# cleaning

In [1]:
import requests 
import json
import os
from getpass import getpass
import pandas as pd
from pandas import json_normalize
#import tweepy
import time
from datetime import datetime
import sys
sys.path.append('../')
import os
from src import functions as fn 
from riotwatcher import LolWatcher, ApiError
from datetime import time
from datetime import datetime


In [2]:
df = pd.read_csv("../data/player_data.csv")

In [10]:
pd.set_option("display.max_columns", 108)
df.sample(4)

,gameDuration,gameMode,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
8,1609061,ARAM,27,0,1,27901,18,53,Blitzcrank,0,2,2848,2848,2848,38044,11,0,1,0,True,False,False,False,False,False,15601,13350,Invalid,0,1,0,6656,3157,3001,3111,4629,3801,2052,18,3,9,MIDDLE,0,2,2,241,37213,21216,15502,0,0,0,1,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",9452,4491,15922,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SOLO,0,57,44,46,39,33,32,7,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,100,NaN,71,1600,47836,26693,0,34554,3393,0,14,222,312,1,0,1170,985,3130,2,3,2,0,0,0,0,0,True
2,1125748,ARAM,14,0,0,18332,17,53,Blitzcrank,0,2,0,0,0,18471,12,0,1,0,True,False,False,False,False,True,13011,12100,Invalid,0,0,1,6655,3157,3020,3102,1058,1058,2052,16,3,10,NONE,0,2,2,134,30188,16192,9091,0,0,0,0,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",3296,1473,13122,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,34,21,23,18,5,4,26,32,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,100,NaN,31,1117,37015,18515,0,22780,1740,0,22,80,284,1,0,3530,850,566,0,0,4,0,0,0,0,0,False
10,1598082,CLASSIC,11,0,1,11127,13,236,Lucian,0,2,3112,17455,3112,3725,1,0,2,1,False,False,False,False,False,True,12557,9950,BOTTOM,2,2,0,6672,0,3006,6676,1053,1043,3340,16,1,8,BOTTOM,492,7,3,1180,4438,1780,3138,4,0,0,0,0,0,9,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",97885,10934,6150,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,CARRY,0,36,28,28,9,3,7,2,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,200,BOTTOM,1,1589,115366,13606,0,9563,2535,555,169,15,36,3,1,13042,891,274,1,2,1,0,18,0,4,8,True
19,1515366,ARAM,18,0,3,27487,18,10,Kayle,0,2,5812,5812,5812,25728,8,0,1,0,False,False,False,False,False,False,18612,16000,Invalid,1,1,1,6673,3006,3091,3094,3153,3085,2052,21,3,9,MIDDLE,500,3,2,215,71163,12540,18362,0,0,0,1,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",108095,15747,13787,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,54,63,40,8,6,21,5,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,404,jalex141,False,200,NaN,3,1507,180804,28287,0,34034,7346,2249,162,52,206,5,0,1545,0,1883,4,4,2,0,0,0,0,0,True


In [17]:
dfr = pd.read_csv("../data/player_data_r.csv")
pd.set_option("display.max_columns", 108)

In [18]:
dfr.head(4)

,gameDuration,gameMode,gameStartTimestamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,1966152,CLASSIC,1629403315758,10,0,5,13029,15,111,Nautilus,0,2,1519,4228,1519,24684,4,0,1,0,False,False,False,False,False,False,10156,8100,UTILITY,0,1.0,0,3860,2031,3047,3068,3075,1033,3364,20,1,6,BOTTOM,0,5,2,416,28820,8422,7118,2,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",15051,3755,6977,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,34,37,28,9,7,14,6,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,62,1958,50107,13051,0,14535,3462,0,48,274,83,1,0,6235,873,439,1,4.0,5,0,40,0,4,18,True
1,1728911,CLASSIC,1629047073458,17,0,0,12454,14,25,Morgana,0,3,1540,5893,1540,7685,4,0,1,0,True,False,False,False,False,False,10018,9000,UTILITY,1,1.0,0,3853,2421,3020,3165,6653,3191,3364,19,1,6,BOTTOM,0,5,2,1151,53360,15683,7646,4,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",5407,658,9083,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,62,79,19,7,6,14,4,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,50,1723,59212,16785,515,16730,4251,0,34,78,114,1,0,444,444,0,0,2.0,0,0,39,0,2,19,True
2,2404409,CLASSIC,1629044262711,16,0,1,17218,17,50,Swain,0,3,3328,6371,3328,26166,10,0,1,0,False,False,False,False,False,False,14976,12950,UTILITY,0,1.0,2,3853,3157,3020,4637,6653,3165,3364,25,4,10,BOTTOM,0,2,2,742,137993,57512,19307,4,0,1,0.0,0,0,5,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",9191,1820,24671,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,109,77,96,24,9,14,6,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,100,UTILITY,83,2396,150400,62336,0,46252,11769,0,89,389,411,1,0,3215,3004,2273,1,5.0,9,0,37,0,2,17,False
3,1966318,CLASSIC,1629041965113,18,0,0,13136,15,111,Nautilus,0,2,1405,5470,1405,39491,4,0,0,0,False,False,False,True,False,False,9657,8400,UTILITY,1,1.0,0,3860,2003,3047,3068,3075,1057,3364,17,0,2,BOTTOM,19,0,1,907,27347,8215,6353,0,0,0,1.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",13439,2709,10049,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,30,37,26,10,6,14,4,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,398,jalex141,False,200,UTILITY,72,1958,48077,11967,0,17404,1931,0,48,287,146,1,0,7290,1042,1001,1,3.0,3,0,46,0,5,22,True


In [10]:
print(datetime.fromtimestamp(1631560758))
datetime.fromtimestamp(1631560758.474).strftime("%H")

2021-09-13 21:19:18


'21'

In [7]:
timestamp = 1631560758
date_time = datetime.fromtimestamp(timestamp)

print("Date time object:", date_time)

d = date_time.strftime("%m/%d/%Y, %H:%M:%S")
print("Output 2:", d)


Date time object: 2021-09-13 21:19:18
Output 2: 09/13/2021, 21:19:18


In [14]:
d

'09/13/2021, 21:19:18'

In [4]:
def duration(mili):
    mili = int(mili/1000)
    sec = mili % 60; mili = mili//60
    minutes = mili % 60; mili = mili//60
    h = mili
    #time_ = (h,minutes,sec)
    date_time = time(h,minutes,sec)
    date_time = date_time.strftime("%H:%M:%S")
    return date_time

In [9]:
datetime.time(date_time)

datetime.time(21, 19, 18)

In [10]:
duration(1966152)

'00:32:46'

In [19]:
#["game_start"] .strftime("%m/%d/%Y, %H:%M:%S") 
dfr.gameStartTimestamp = dfr.gameStartTimestamp.apply(lambda x: datetime.fromtimestamp(x/1000).strftime("%H"))
dfr.gameDuration = dfr.gameDuration.apply(duration)
dfr.timePlayed = dfr.timePlayed.apply(lambda x: x/60)

dfr.head(5)

,gameDuration,gameMode,gameStartTimestamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,00:32:46,CLASSIC,22,10,0,5,13029,15,111,Nautilus,0,2,1519,4228,1519,24684,4,0,1,0,False,False,False,False,False,False,10156,8100,UTILITY,0,1.0,0,3860,2031,3047,3068,3075,1033,3364,20,1,6,BOTTOM,0,5,2,416,28820,8422,7118,2,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",15051,3755,6977,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,34,37,28,9,7,14,6,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,62,32.633333,50107,13051,0,14535,3462,0,48,274,83,1,0,6235,873,439,1,4.0,5,0,40,0,4,18,True
1,00:28:48,CLASSIC,19,17,0,0,12454,14,25,Morgana,0,3,1540,5893,1540,7685,4,0,1,0,True,False,False,False,False,False,10018,9000,UTILITY,1,1.0,0,3853,2421,3020,3165,6653,3191,3364,19,1,6,BOTTOM,0,5,2,1151,53360,15683,7646,4,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",5407,658,9083,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,62,79,19,7,6,14,4,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,50,28.716667,59212,16785,515,16730,4251,0,34,78,114,1,0,444,444,0,0,2.0,0,0,39,0,2,19,True
2,00:40:04,CLASSIC,18,16,0,1,17218,17,50,Swain,0,3,3328,6371,3328,26166,10,0,1,0,False,False,False,False,False,False,14976,12950,UTILITY,0,1.0,2,3853,3157,3020,4637,6653,3165,3364,25,4,10,BOTTOM,0,2,2,742,137993,57512,19307,4,0,1,0.0,0,0,5,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",9191,1820,24671,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,109,77,96,24,9,14,6,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,100,UTILITY,83,39.933333,150400,62336,0,46252,11769,0,89,389,411,1,0,3215,3004,2273,1,5.0,9,0,37,0,2,17,False
3,00:32:46,CLASSIC,17,18,0,0,13136,15,111,Nautilus,0,2,1405,5470,1405,39491,4,0,0,0,False,False,False,True,False,False,9657,8400,UTILITY,1,1.0,0,3860,2003,3047,3068,3075,1057,3364,17,0,2,BOTTOM,19,0,1,907,27347,8215,6353,0,0,0,1.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",13439,2709,10049,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,30,37,26,10,6,14,4,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,398,jalex141,False,200,UTILITY,72,32.633333,48077,11967,0,17404,1931,0,48,287,146,1,0,7290,1042,1001,1,3.0,3,0,46,0,5,22,True
4,00:31:46,CLASSIC,17,13,0,0,10667,13,57,Maokai,0,2,1783,1852,1783,12852,6,0,0,0,False,Fa

In [39]:
dfr.shape

(50, 108)

In [21]:
dfr.groupby(['role','teamPosition','championName']).size()

role     teamPosition  championName
CARRY    BOTTOM        Ashe             1
                       Lucian           2
DUO      TOP           DrMundo          1
NONE     JUNGLE        Sejuani          1
         TOP           Poppy            1
         UTILITY       Maokai           1
SOLO     MIDDLE        Galio            1
         TOP           DrMundo          1
                       Sett             1
                       Yorick           1
SUPPORT  TOP           Sett             1
         UTILITY       Alistar          2
                       Bard             1
                       Lux              2
                       Maokai           8
                       Morgana          4
                       Nautilus        11
                       Poppy            1
                       Swain            7
                       Xerath           1
                       Ziggs            1
dtype: int64

In [22]:
dfr.role.value_counts().index[0]

'SUPPORT'

In [22]:
interesting_columns = ['gameStartTimestamp','assists','champExperience','damageDealtToBuildings','damageDealtToObjectives','damageDealtToTurrets','damageSelfMitigated','deaths','firstBloodAssist','firstBloodKill','goldEarned','inhibitorTakedowns','kills','timeCCingOthers','timePlayed','totalDamageDealt','totalDamageDealtToChampions','totalMinionsKilled','totalTimeSpentDead','visionScore','win']

In [21]:
dfr_f = dfr[dfr.role == dfr.role.value_counts().index[0]]
dfr_f.head(5)

,gameDuration,gameMode,gameStartTimestamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,00:32:46,CLASSIC,22,10,0,5,13029,15,111,Nautilus,0,2,1519,4228,1519,24684,4,0,1,0,False,False,False,False,False,False,10156,8100,UTILITY,0,1.0,0,3860,2031,3047,3068,3075,1033,3364,20,1,6,BOTTOM,0,5,2,416,28820,8422,7118,2,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",15051,3755,6977,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,34,37,28,9,7,14,6,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,62,32.633333,50107,13051,0,14535,3462,0,48,274,83,1,0,6235,873,439,1,4.0,5,0,40,0,4,18,True
1,00:28:48,CLASSIC,19,17,0,0,12454,14,25,Morgana,0,3,1540,5893,1540,7685,4,0,1,0,True,False,False,False,False,False,10018,9000,UTILITY,1,1.0,0,3853,2421,3020,3165,6653,3191,3364,19,1,6,BOTTOM,0,5,2,1151,53360,15683,7646,4,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",5407,658,9083,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,62,79,19,7,6,14,4,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,50,28.716667,59212,16785,515,16730,4251,0,34,78,114,1,0,444,444,0,0,2.0,0,0,39,0,2,19,True
2,00:40:04,CLASSIC,18,16,0,1,17218,17,50,Swain,0,3,3328,6371,3328,26166,10,0,1,0,False,False,False,False,False,False,14976,12950,UTILITY,0,1.0,2,3853,3157,3020,4637,6653,3165,3364,25,4,10,BOTTOM,0,2,2,742,137993,57512,19307,4,0,1,0.0,0,0,5,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",9191,1820,24671,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,109,77,96,24,9,14,6,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,100,UTILITY,83,39.933333,150400,62336,0,46252,11769,0,89,389,411,1,0,3215,3004,2273,1,5.0,9,0,37,0,2,17,False
3,00:32:46,CLASSIC,17,18,0,0,13136,15,111,Nautilus,0,2,1405,5470,1405,39491,4,0,0,0,False,False,False,True,False,False,9657,8400,UTILITY,1,1.0,0,3860,2003,3047,3068,3075,1057,3364,17,0,2,BOTTOM,19,0,1,907,27347,8215,6353,0,0,0,1.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",13439,2709,10049,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,30,37,26,10,6,14,4,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,398,jalex141,False,200,UTILITY,72,32.633333,48077,11967,0,17404,1931,0,48,287,146,1,0,7290,1042,1001,1,3.0,3,0,46,0,5,22,True
4,00:31:46,CLASSIC,17,13,0,0,10667,13,57,Maokai,0,2,1783,1852,1783,12852,6,0,0,0,False,Fa

In [23]:
dfr_fr = dfr_f[interesting_columns]
dfr_fr.head(3)

,gameStartTimestamp,assists,champExperience,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,firstBloodAssist,firstBloodKill,goldEarned,inhibitorTakedowns,kills,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalMinionsKilled,totalTimeSpentDead,visionScore,win
0,22,10,13029,1519,4228,1519,24684,4,False,False,10156,1.0,6,62,32.633333,50107,13051,48,83,40,True
1,19,17,12454,1540,5893,1540,7685,4,True,False,10018,1.0,6,50,28.716667,59212,16785,34,114,39,True
2,18,16,17218,3328,6371,3328,26166,10,False,False,14976,1.0,10,83,39.933333,150400,62336,89,411,37,False


In [34]:
time_cols = ["assists","champExperience","damageDealtToBuildings","damageDealtToObjectives","damageSelfMitigated","deaths","goldEarned","kills","timeCCingOthers","totalDamageDealt","totalDamageDealtToChampions","totalMinionsKilled","totalTimeSpentDead","visionScore"]
count_cols = ["gameStartTimestamp"]
percent_cols = ["firstBloodAssist","firstBloodKill","inhibitorTakedowns","win"]

In [ ]:
for col in time_cols:
    dfr_fr[col] = dfr_fr[col]/dfr_fr.timePlayed

In [26]:
dfr_fr.head(3)

,gameStartTimestamp,assists,champExperience,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,firstBloodAssist,firstBloodKill,goldEarned,inhibitorTakedowns,kills,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalMinionsKilled,totalTimeSpentDead,visionScore,win
0,22,0.306435,399.254341,46.547497,129.560776,1519,756.404494,0.122574,False,False,311.215526,1.0,0.183861,1.899898,32.633333,1535.454545,399.928498,1.470889,2.543412,1.225741,True
1,19,0.591991,433.685432,53.627394,205.211840,1540,267.614626,0.139292,True,False,348.856645,1.0,0.208938,1.741149,28.716667,2061.938479,584.503772,1.183981,3.969820,1.358096,True
2,18,0.400668,431.168614,83.338898,159.540902,3328,655.242070,0.250417,False,False,375.025042,1.0,0.250417,2.078464,39.933333,3766.277129,1561.001669,2.228715,10.292154,0.926544,False


In [32]:
dfr_fr.gameStartTimestamp.value_counts().index[0]

'20'

In [36]:
len(dfr_fr["timePlayed"])

43

In [41]:
resume = {}
for col in time_cols:
    resume[col] = dfr_fr[col].median()
for col in count_cols:
    resume[col] = int(dfr_fr[col].value_counts().index[0])
    [0]
for col in percent_cols:
    resume[col] = dfr_fr[col].sum()/len(dfr_fr[col])
resume["timePlayed"] = dfr_fr.timePlayed.sum()/len(dfr_fr[col])
resume

{'assists': 0.4006677796327212,
 'champExperience': 399.25434116445354,
 'damageDealtToBuildings': 40.362426035502956,
 'damageDealtToObjectives': 119.42666029622552,
 'damageSelfMitigated': 536.1002349256069,
 'deaths': 0.17686072218128224,
 'goldEarned': 311.2155260469867,
 'kills': 0.11406844106463877,
 'timeCCingOthers': 1.8323153803442533,
 'totalDamageDealt': 1994.4974044360545,
 'totalDamageDealtToChampions': 584.5037724898433,
 'totalMinionsKilled': 1.5502055196711686,
 'totalTimeSpentDead': 5.166561910747958,
 'visionScore': 1.3686806411837238,
 'gameStartTimestamp': 20,
 'firstBloodAssist': 0.13953488372093023,
 'firstBloodKill': 0.023255813953488372,
 'inhibitorTakedowns': 0.09302325581395349,
 'timePlayed': 32.32906976744185}

In [49]:
pd.DataFrame([resume])

,assists,champExperience,damageDealtToBuildings,damageDealtToObjectives,damageSelfMitigated,deaths,goldEarned,kills,timeCCingOthers,totalDamageDealt,totalDamageDealtToChampions,totalMinionsKilled,totalTimeSpentDead,visionScore,gameStartTimestamp,firstBloodAssist,firstBloodKill,inhibitorTakedowns,timePlayed
0,0.400668,399.254341,40.362426,119.42666,536.100235,0.176861,311.215526,0.114068,1.832315,1994.497404,584.503772,1.550206,5.166562,1.368681,20,0.139535,0.023256,0.093023,32.32907
